# 🔄 QEPC Data Refresh

**Update your data from NBA APIs**

⚠️ **Run on LOCAL machine only** (not iPad/cloud)

This notebook fetches:
- 📊 Current team ratings (ORtg, DRtg, Pace)
- 📅 Today's game schedule
- 💰 **Vegas betting odds** (NEW!)
- 🏥 Injury report

---

In [ ]:
# SETUP
import sys
from pathlib import Path
from datetime import datetime
import time
import json

# Your paths
project_root = Path(r"C:\Users\wdors\qepc_project")
qepc_v2_path = project_root / "experimental" / "CLAUDE_REWRITE" / "qepc_v2"
data_dir = project_root / "data"

# Add to path
if str(qepc_v2_path) not in sys.path:
    sys.path.insert(0, str(qepc_v2_path))

print(f"📁 Project root: {project_root}")
print(f"📂 Data directory: {data_dir}")

# Check for required packages
try:
    import requests
    print("✅ requests available")
except ImportError:
    print("❌ requests not installed: pip install requests")

try:
    from nba_api.stats.endpoints import leaguedashteamstats
    HAS_NBA_API = True
    print("✅ nba_api available")
except ImportError:
    HAS_NBA_API = False
    print("⚠️  nba_api not installed (optional): pip install nba_api")

import pandas as pd
print("\n✅ Ready to refresh!")

---
## 📅 Fetch Today's Games (NBA Live API)

In [ ]:
print("📅 Fetching today's games...")

url = "https://cdn.nba.com/static/json/liveData/scoreboard/todaysScoreboard_00.json"
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Referer': 'https://www.nba.com/'
}

try:
    response = requests.get(url, headers=headers, timeout=10)
    
    if response.status_code == 200:
        data = response.json()
        games = data['scoreboard'].get('games', [])
        game_date = data['scoreboard'].get('gameDate', 'Unknown')
        
        print(f"✅ Found {len(games)} games on {game_date}")
        
        # Parse into DataFrame
        rows = []
        for g in games:
            home = g.get('homeTeam', {})
            away = g.get('awayTeam', {})
            rows.append({
                'game_id': g.get('gameId'),
                'game_date': game_date,
                'game_status': g.get('gameStatusText'),
                'home_team': f"{home.get('teamCity', '')} {home.get('teamName', '')}".strip(),
                'home_score': home.get('score', 0),
                'away_team': f"{away.get('teamCity', '')} {away.get('teamName', '')}".strip(),
                'away_score': away.get('score', 0),
            })
        
        todays_games = pd.DataFrame(rows)
        
        # Save
        output_path = data_dir / "live" / "todays_games.csv"
        output_path.parent.mkdir(parents=True, exist_ok=True)
        todays_games.to_csv(output_path, index=False)
        print(f"💾 Saved to: {output_path.name}")
        
        display(todays_games)
    else:
        print(f"❌ HTTP {response.status_code}")
except Exception as e:
    print(f"❌ Error: {e}")

---
## 💰 Fetch Vegas Odds (NBA Live API)

In [ ]:
print("💰 Fetching Vegas odds...")

url = "https://cdn.nba.com/static/json/liveData/odds/odds_todaysGames.json"

try:
    response = requests.get(url, headers=headers, timeout=10)
    
    if response.status_code == 200:
        data = response.json()
        games = data.get('games', [])
        
        print(f"✅ Found odds for {len(games)} games")
        
        # Parse odds
        rows = []
        for g in games:
            game_id = g.get('gameId')
            home_id = g.get('homeTeamId')
            away_id = g.get('awayTeamId')
            
            spread_home = None
            ml_home = None
            ml_away = None
            
            for market in g.get('markets', []):
                books = market.get('books', [])
                # Prefer US books
                us_books = [b for b in books if b.get('countryCode') == 'US']
                book = us_books[0] if us_books else (books[0] if books else None)
                
                if book:
                    for outcome in book.get('outcomes', []):
                        if market.get('name') == '2way':
                            odds = float(outcome.get('odds', 0))
                            if outcome.get('type') == 'home':
                                ml_home = odds
                            elif outcome.get('type') == 'away':
                                ml_away = odds
                        
                        elif market.get('name') == 'spread':
                            spread = outcome.get('spread')
                            if spread and outcome.get('type') == 'home':
                                spread_home = float(spread)
            
            # Convert ML to implied probability
            home_prob = 1/ml_home if ml_home and ml_home > 0 else None
            away_prob = 1/ml_away if ml_away and ml_away > 0 else None
            
            rows.append({
                'game_id': game_id,
                'home_team_id': home_id,
                'away_team_id': away_id,
                'vegas_spread_home': spread_home,
                'vegas_ml_home': ml_home,
                'vegas_ml_away': ml_away,
                'vegas_implied_home_prob': home_prob,
                'vegas_implied_away_prob': away_prob,
            })
        
        odds_df = pd.DataFrame(rows)
        
        # Save
        output_path = data_dir / "live" / "todays_odds.csv"
        odds_df.to_csv(output_path, index=False)
        print(f"💾 Saved to: {output_path.name}")
        
        display(odds_df)
    else:
        print(f"❌ HTTP {response.status_code}")
except Exception as e:
    print(f"❌ Error: {e}")

---
## 📊 Fetch Team Ratings (NBA Stats API)

In [ ]:
if HAS_NBA_API:
    print("📊 Fetching team ratings...")
    
    # Get current season
    now = datetime.now()
    year = now.year if now.month >= 10 else now.year - 1
    season = f"{year}-{str(year + 1)[2:]}"
    print(f"   Season: {season}")
    
    try:
        time.sleep(1)  # Be nice to API
        
        stats = leaguedashteamstats.LeagueDashTeamStats(
            season=season,
            measure_type_detailed_defense='Advanced',
            per_mode_detailed='PerGame'
        )
        
        df = stats.get_data_frames()[0]
        
        # Select columns
        ratings = df[['TEAM_ID', 'TEAM_NAME', 'W', 'L', 'GP',
                     'OFF_RATING', 'DEF_RATING', 'NET_RATING', 'PACE']].copy()
        
        ratings = ratings.rename(columns={
            'TEAM_ID': 'team_id',
            'TEAM_NAME': 'Team',
            'W': 'Wins',
            'L': 'Losses',
            'GP': 'GamesPlayed',
            'OFF_RATING': 'ORtg',
            'DEF_RATING': 'DRtg',
            'NET_RATING': 'NetRtg',
            'PACE': 'Pace'
        })
        
        ratings['Season'] = season
        ratings['updated_at'] = datetime.now().isoformat()
        
        # Save
        output_path = data_dir / "live" / "team_ratings.csv"
        ratings.to_csv(output_path, index=False)
        print(f"✅ Saved {len(ratings)} teams to: {output_path.name}")
        
        display(ratings.head(10))
        
    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("⚠️  Skipping team ratings - nba_api not installed")
    print("   Install with: pip install nba_api")

---
## ✅ Summary

In [ ]:
print("\n" + "=" * 50)
print("📦 DATA REFRESH COMPLETE")
print("=" * 50)

# Show what was updated
live_dir = data_dir / "live"
if live_dir.exists():
    print(f"\n📁 Files in live/:")
    for f in sorted(live_dir.glob('*.csv')):
        size = f.stat().st_size / 1024
        mtime = datetime.fromtimestamp(f.stat().st_mtime)
        print(f"   {f.name:30} {size:6.1f} KB  (updated: {mtime:%H:%M})")

print("\n🏀 Ready to run predictions!")

---

## 📋 What Each File Contains

| File | Source | Contents |
|------|--------|----------|
| `todays_games.csv` | NBA Live API | Today's schedule, scores |
| `todays_odds.csv` | NBA Live API | Vegas spreads, moneylines |
| `team_ratings.csv` | NBA Stats API | ORtg, DRtg, Pace, NetRtg |

Run this notebook **before** making predictions to get the latest data!